# Combine results from different fits

Having fitted a set of speed factors from different but uniform samples, and knowing that the absolute scale is not determined, before comparing values from different samples it is necessary to rescale them to the same scale. My idea is to minimise the modulus of the difference between the first vector and the N-th vector multiplied by a factor, and rescale it accordingly.

In [14]:
import math
import csv
from collections import defaultdict
import numpy as np

from __future__ import division

%matplotlib inline

Let's load in memory the test results from different files (assuming they are homogeneous, i.e. they are different measurements of the same quantities under the same conditions).

In [15]:
n_files = 10
fact = dict()
factors = defaultdict(dict)   # Dict containing all k_factors fitted for a given CPU, indexed by cpu type and file number
factors_err = defaultdict(dict)   # Dict containing all k_factors fitted for a given CPU, indexed by cpu type and file number
for i in range(n_files):
    input_pattern = 'results_cputype_task_cpu_sub_%s_pile_0.001.csv'
    f = open(input_pattern % (str(i)))
    for line in f:
        cpu, k, k_err = line.strip().split(',')
        k = float(k)
        factors[cpu][i] = k
        factors_err[cpu][i] = k_err
    f.close()

A series of "rescaling functions" are defined and tested to see how much they differ in their effects.

In [16]:
def dot(x, y):   # a special inner product that skips components where one of the vectors has a NaN
    result = 0.
    for i in range(len(x)):
        if not np.isnan(x[i]) and not np.isnan(y[i]):
            result += x[i] * y[i]
    return result

In [17]:
def rescale_dist(x, y):   # rescales y such that the distance to x is minimised
    y2 = y * x / x   # to put nan in elements where x has NaN
    alpha = np.nansum(x * y) / np.nansum(y2 * y2)
    return y * alpha

In [18]:
def rescale_first(x, y):   # rescales y such that it has the first element equal to x's one
    return y / y[0] * x[0]

In [19]:
def rescale_mod(x):   # rescales x such that its Euclidean norm is one (NaN components are ignored)
    n = len(x) - np.isnan(x).sum()
    s = math.sqrt(np.nansum(x * x))
    return x / s * math.sqrt(n)

In [20]:
def rescale_1norm(x):   # rescales x such that its l-1 norm is one 
    n = len(x) - np.isnan(x).sum()
    s = np.nansum(np.fabs(x))
    return x / s * n

Let's put the numbers in a matrix b where rows correspond to files and CPUs (mapped to numbers) to columns.

In [21]:
cmap = dict()   # mapping each CPU type to an integer starting from zero
i = -1
a = np.zeros((len(factors), n_files))   # a matrix with as many rows as CPU types and columns as files
for c in factors.keys():
    i += 1
    cmap[c] = i
    a[i] = [factors[c].setdefault(l, np.nan) for l in range(n_files)]   # a NaN is used if the CPU type is not present in file
b = a.transpose()   # rows are files and columns are speed factors

Let's use as "reference" a vector where all CPU factors are one.

In [22]:
ref = np.ones(len(factors))

Let's convert the speed factors according to different methods to make them "averageable".

In [23]:
b1 = np.empty_like(b)
b2 = np.empty_like(b)
b3 = np.empty_like(b)
b4 = np.empty_like(b)
b5 = np.empty_like(b)
for i in range(n_files):
    v = b[i]   # speed factors for file i
    b1[i] = rescale_dist(ref, v)   
    b2[i] = rescale_first(b[0], v)   # the first factor of the first file is used as reference (AV's suggestion)
    b3[i] = rescale_mod(v)
    b4[i] = rescale_dist(b[0], v)   # AS's initial idea
    b5[i] = rescale_1norm(v)
a1 = b1.transpose()
a2 = b2.transpose()
a3 = b3.transpose()
a4 = b4.transpose()
a5 = b5.transpose()

Let's visually inspect how the RMSes of the rescaled values for all files compare, for different rescaling methods.

In [24]:
for i in range(len(a)):
    print '%.4f %.4f %.4f %.4f %.4f %.4f' % (
        np.nanstd(a[i]) / np.nanmean(a[i]),
        np.nanstd(a1[i]) / np.nanmean(a1[i]),
        np.nanstd(a2[i]) / np.nanmean(a2[i]),
        np.nanstd(a3[i]) / np.nanmean(a3[i]),
        np.nanstd(a4[i]) / np.nanmean(a4[i]),
        np.nanstd(a5[i]) / np.nanmean(a5[i])
        )

0.0140 0.0119 0.0000 0.0125 0.0119 0.0131
0.0017 0.0062 0.0131 0.0059 0.0033 0.0057
0.0115 0.0118 0.0054 0.0122 0.0097 0.0126
0.0112 0.0121 0.0109 0.0124 0.0110 0.0127
0.0000 0.0000 nan 0.0000 0.0000 0.0000
0.0101 0.0098 0.0164 0.0096 0.0093 0.0094
0.0075 0.0112 0.0151 0.0111 0.0069 0.0109
0.0054 0.0066 0.0160 0.0064 0.0065 0.0062
0.0102 0.0106 0.0082 0.0107 0.0085 0.0108
0.0228 0.0239 0.0353 0.0235 0.0237 0.0230
0.0070 0.0071 0.0156 0.0069 0.0055 0.0068
0.0057 0.0074 0.0126 0.0073 0.0037 0.0072
0.0068 0.0092 0.0181 0.0087 0.0075 0.0082
0.0042 0.0076 0.0149 0.0072 0.0044 0.0069
0.0089 0.0086 0.0153 0.0085 0.0070 0.0084
0.0112 0.0127 0.0168 0.0126 0.0091 0.0125
0.0114 0.0110 0.0181 0.0108 0.0098 0.0106
0.0053 0.0080 0.0147 0.0077 0.0058 0.0075
0.0071 0.0081 0.0195 0.0079 0.0088 0.0078
0.0451 0.0416 0.0000 0.0419 0.0436 0.0422
0.0105 0.0110 0.0175 0.0111 0.0079 0.0113
0.0088 0.0088 0.0182 0.0086 0.0087 0.0086
0.0079 0.0084 0.0181 0.0080 0.0093 0.0077
0.0179 0.0152 0.0086 0.0155 0.0160 0.

This shows that any rescaling method strongly reduces the RMS, which is consistent with the fact that the speed factors fitted on different values don't have a priori the same scale. Therefore, rescaling is absolutely necessary if we want to combine different measurements.

Another way to see this is to calculate the sum of all the normalised rms and compare the totals.

In [25]:
t = 0.
t1 = 0.
t2 = 0.
t3 = 0.
t4 = 0.
t5 = 0.

for i in range(len(a)):
    t += np.nanstd(a[i]) / np.nanmean(a[i])
    t1 += np.nanstd(a1[i]) / np.nanmean(a1[i])
    t2 += np.nanstd(a2[i]) / np.nanmean(a2[i])
    t3 += np.nanstd(a3[i]) / np.nanmean(a3[i])
    t4 += np.nanstd(a4[i]) / np.nanmean(a4[i])
    t5 += np.nanstd(a4[i]) / np.nanmean(a5[i])
print t, t1, t2, t3, t4, t5

0.881761769152 0.953407510742 nan 0.948501484745 0.835977587236 0.822494743494


These numbers show that all rescaling methods give similarly sized rms.

Finally, let's actually calculate the combined speed factors.

In [26]:
output_file = 'results_cputype_task_cpu_pile_0.001.csv'
with open(output_file, 'wb') as csvfile:
    w = csv.writer(csvfile, delimiter=',')
    for s in sorted(cmap.keys()):
        print '%.4f' % np.nanmean(a5[cmap[s]])
        w.writerow([s, np.nanmean(a5[cmap[s]])])

0.8554
0.6721
0.7550
0.6497
0.6564
0.6764
0.6375
0.6528
0.6809
0.9496
0.7597
0.7920
0.8168
1.1673
0.8529
1.0120
1.0624
1.1172
1.1652
0.9009
0.8551
0.7404
1.0966
0.8504
0.8795
0.8902
1.1567
1.0240
0.9361
0.7303
0.9481
1.2357
1.2874
1.0055
0.9616
0.9284
0.9873
1.3556
1.0464
1.1211
0.8262
1.1376
0.9939
1.1510
0.8773
1.0115
1.0769
1.1840
1.2002
1.1222
0.9170
1.0962
1.0077
1.0271
1.4869
1.2037
0.9587
1.0099
0.9252
1.1515
1.0307
1.0577
1.0363
1.0952
1.0175
1.1299
1.2519
1.4671
1.1806
1.1756
1.3810
1.6621
1.2656
1.1941
1.0662
0.6803
0.8732
